In [1]:
from PIL import Image
import os
import csv
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader

In [3]:
def test_model_and_save_results(model, test_dir, output_csv):
    model.eval()  # Set the model to evaluation mode

    results = []  # To store results for the CSV

    # Disable gradient computation during inference
    with torch.no_grad():
        for file_name in os.listdir(test_dir):
            file_path = os.path.join(test_dir, file_name)

            # Check if the file is an image
            if not file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue

            # Load and preprocess the image
            image = Image.open(file_path).convert("RGB")
            processed_image = test_transform(image).unsqueeze(0)

            # Perform inference
            # The model outputs a value between -1 and 1
            outputs = model(processed_image)
            # We need a probability score, this is why we apply sigmoid to outputs
            # We assume that if score >0.5 then the image contains a fire
            # IMPORTANT: the model outputs a logi between -1 and 1 
            # in this case the model outputs how likeley a picture is to 
            # NOT have a fire ( ~ -1: fire , ~ 1: NO fire )
            preds = torch.sigmoid(outputs) < 0.5  
            predicted_class = int(preds.item())  # Convert to 0 or 1

            # Append result
            results.append({'id': file_name, 'class': predicted_class})
            
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['id', 'class'])
        writer.writeheader()
        writer.writerows(results)

    print(f"Results saved to {output_csv}")

In [4]:
# PREPARE THE PARAMETERS FOR TEST FUNCTION

# Define the model structure
def get_model_resnet50():
    model = models.resnet50(pretrained=True)  # Load pretrained ResNet18
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 1)  # Modify the last layer for binary classification
    return model
    
# Import the model trained on kaggle
model = get_model_resnet50()
#model = get_model_resnet18()
model.load_state_dict(torch.load("/home/leyla/ChallengeDL/best_fire_classifier.pth",
                    map_location=torch.device('cpu'), weights_only=True))

# Define the transformations to pre-process test data, 
# Make sure it's the same transformation as for validation and training
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Path to test dir
test_dir = "dl2425_challenge_dataset/test"

# output csv file path
output_csv = "testv2.csv"

/home/leyla/.pyenv/versions/d2l-env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/leyla/.pyenv/versions/d2l-env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/leyla/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100.0%


RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "fc.weight", "fc.bias". 
	Unexpected key(s) in state_dict: "fc.1.weight", "fc.1.bias". 

In [5]:
state_dict = torch.load(
    "/home/leyla/ChallengeDL/best_fire_classifier.pth", map_location=torch.device('cpu'))
print(state_dict.keys())

/tmp/ipykernel_30556/1728326529.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.0.conv3.weight', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.bn3.num_batches_tracked', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.0.downsample.1.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.we

In [ ]:
test_model_and_save_results(model, test_dir, output_csv)

In [ ]:
# Use this to empty a csv file
with open(output_csv, 'w'):
    pass  # This will truncate the file to 0 length, removing all its content